In [55]:
from socket import *

serverName = 'snoopy.mpi-inf.mpg.de'
serverPort = 666

# TCP connection setup
clientSocket = socket(AF_INET, SOCK_STREAM)
clientSocket.connect((serverName, serverPort))

# Server reply
serverStatus = clientSocket.recv(1024)
print(serverStatus.decode('utf-8'))

# Client HELLO reply
hello = "HELLO.\n".encode('utf-8')
clientSocket.sendall(hello)
print(hello.decode())

# Server COMMAND reply
serverStatus = clientSocket.recv(1024)
print(serverStatus.decode())

# Client COMMAND reply
cmd = "DOWNLOAD.\n".encode('utf-8')
clientSocket.sendall(cmd)
print(cmd.decode())

# Server filename and filesize replies
fname = bytearray(clientSocket.recv(18))
filesize = clientSocket.recv(22)
print(fname.decode())
print(filesize.decode())

# Extracting filesize
sizeBegin = filesize.index(bytearray(":", "utf-8")) + 1
sizeEnd = filesize.index(bytearray("BYTES", "utf-8")) - 1
filesize = int(filesize[sizeBegin:sizeEnd])


#initializations for loop
byeReqd = bytearray("BYE.", 'utf-8')
tokReqd = bytearray("TOKEN:", 'utf-8')
token = bytearray('', 'utf-8')
bye = bytearray('', 'utf-8')
imgByteArray = bytearray('', 'utf-8')
tokindex = 0 
byeindex = 0

# while bye not received feed data into image bytearray
while bye != byeReqd:
    
    msg = bytearray(clientSocket.recv(4096))
    
    # Extract TOKEN
    if tokReqd in msg:
        tokindex = msg.index(bytearray("TOKEN", 'utf-8'))
    
    # Extact BYE
    if byeReqd in msg:
        byeindex = msg.index(bytearray("BYE", 'utf-8'))
        token = msg[tokindex:(byeindex - 1)]
        bye = msg[byeindex:-1]
        del msg[tokindex:-1:1]
        
    imgByteArray += msg
    
print(token.decode())
print(bye.decode())
print("Image recieved.")

# TCP connection turned off upon indication by server
if reqd in bye:
    clientSocket.close()
    print("\n")
    print("Caught the cat!!")

# filename extraction
nameBegin = fname.index(bytearray(":","utf-8")) + 2
nameEnd = -1
filename = fname[nameBegin:nameEnd].decode()

# writing the received data into file
file = open(filename, "wb+") 
file.write(imgByteArray)
file.close()

HELLO?

HELLO.

COMMAND?

DOWNLOAD.

FILENAME: CAT.JPG

FILESIZE: 62531 BYTES

TOKEN: 9693f8acfdcda76cde83be0ffaa98de2
BYE.
Image recieved.


Caught the cat!!
